# Introduction

This notebook provides simple implementations of the sigma protocols specified [here](https://github.com/zkpstandard/wg-sigma-protocols/)

TODO get a better link

In [1]:
from hashlib import sha256 as hash_function # not recommended for real implementations

* `prover_commit` generates a commitment `T` and a prover state `pstate`. The prover commits to the secret witness in a way that doesn't reveal anything about the witness itself. Commitment `T` is sent to the verifier, and `pstate` is kept secret.
* `prover_response` provides a proof of knowledge given a challenge.
* `label` is used to identify

In [2]:
class SigmaProtocol:
    def prover_commit(self, witness):
        pass

    def prover_response(self, state, challenge):
        pass
      
    def label(self):
        pass
    
    def simulate_commitment(self, challenge, response):
        pass
    
    def simulate_response(self):
        pass
    
    def verifier(self, commitment, challenge, response):
        pass

TODO: this is the and composition, this is the relation it proves, these are the algorithms used?

In [120]:
class SigmaAndComposition(SigmaProtocol):

    # Left and right are both sigma protocols
    def __init__(self, left: SigmaProtocol, right: SigmaProtocol):
        self.left = left
        self.right = right
        
    def prover_commit(self, witness):
        w0, w1 = witness 
        left_state, left_commitment = self.left.prover_commit(w0)
        right_state, right_commitment = self.right.prover_commit(w1)
        return (left_state, right_state), (left_commitment, right_commitment)
    
    def prover_response(self, state, challenge):
        left_state, right_state = state
        left_response = self.left.prover_response(left_state, challenge)
        right_response = self.right.prover_response(right_state, challenge)
        return (left_response, right_response)
        
    def label(self):
        label = hash_function()
        label.update("zkpstd/sigma/and-v0.0.1")
        label.update(self.left.label())
        label.update(self.right.label())
        return label.digest()
    
    def simulate_commitment(self, challenge, response):
        left_response, right_response = response
        left_commitment = self.left.simulate_commitment(challenge, left_response)
        right_commitment = self.right.simulate_commitment(challenge, right_response)
    
    def simulate_response(self):
        return (self.left.simulate_response(), self.right.simulate_response())

    def verifier(self, commitment, challenge, response):
        left_commitment, right_commitment = commitment
        left_response, right_response = response
        return (self.left.verifier(left_commitment, challenge, left_response)) and \
        (self.right.verifier(right_commitment, challenge, right_response))

These are parameters for `p-521`, one of our standard's supported curves. `Fp` is the scalar field, and `G` is the prime-order group.

In [121]:
p = 0x01ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
K = GF(p)
a = K(0x01fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffc)
b = K(0x0051953eb9618e1c9a1f929a21a0b68540eea2da725b99b315f3b8b489918ef109e156193951ec7e937b1652c0bd3bb1bf073573df883d2c34f1ef451fd46b503f00)
E = EllipticCurve(K, (a, b))
G = E(0x00c6858e06b70404e9cd9e3ecb662395b4429c648139053fb521f828af606b4d3dbaa14b5e77efe75928fe1dc127a2ffa8de3348b3c1856a429bf97e7e31c2e5bd66, 0x011839296a789a3bc0045c8a5fb42c7d1bd998f54449579b446817afbd17273e662c97ee72995ef42640c550b9013fad0761353c7086a272c24088be94769fd16650)
E.set_order(0x01fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffa51868783bf2f966b7fcc0148f709a5d03bb5c9b8899c47aebb6fb71e91386409 * 0x1)

p_521 = {"Fp": K, "E": E, "G": G, "p": p}

In [122]:
class DlogTemplate(SigmaProtocol):
    # n is the input dimension of the homomorphism
    # m is the output dimension of the homomorphism
    n = None
    m = None
    
    # the input type of the homomorphism
    Fp = None
    
    # The prime number defining this field
    p = None
    
    # The output type of the homomorphism
    G = None
    
    # The elliptic curve
    E = None
    
    def __init__(self, pk):
        assert(len(pk) == self.n)
        self.pk = pk
        
    def prover_commit(self, sk):
        assert(len(sk) == self.n)
        # nonce is an n-length array of Fp elements
        nonce = [self.Fp.random_element() for _ in range(self.n)]
        # commitment is an m-length array of G2 elements
        commitment = self._morphism(nonce)
        prover_state = (sk, nonce)
        return (prover_state, commitment)
    
    # Prover_state is nonce (array of n Fp elements) * sk (array of n Fp elements)
    # challenge is an Fp element
    # outputs an array of n Fp elements
    def prover_response(self, prover_state, challenge):
        sk, nonce = prover_state
        return [Integer(challenge) * Integer(sk_i) + Integer(nonce_i) for (sk_i, nonce_i) in zip(sk, nonce)]
    
    def simulate_response(self):
        return [self.Fp.random_element() for _ in range(self.m)]
    
    def simulate_commitment(self, challenge, response):
        return [out_i - pk_i * Integer(challenge) for (out_i, pk_i) in zip(self._morphism(response), self.pk)]

    def _morphism(self, x):
        raise NotImplementedError
        
    def _morphism_label(self):
        raise NonImplementedError
    
    def label(self):
        return self._morphism_label()
    
    def verifier(self, commitment, challenge, response):
        return all(phi_response_i == commitment_i + statement_i * Integer(challenge)
            for phi_response_i, commitment_i, statement_i in zip(self._morphism(response), commitment, self.pk))

In [123]:
class SchnorrDlog(DlogTemplate):
    Fp = p_521["Fp"]
    G = p_521["G"]
    p = p_521["p"]
    E = p_521["E"]
    n = 1
    m = 1
    
    # Inputs an array of one Fp element `x`
    # Outputs `[G * x]`
    def _morphism(self, x):
        return [G * Integer(x[0])]
    
    def _morphism_label(self):
        label = hash_function()
        label.update('schnorr')
        label.update(self.G)
        label.update(self.pk[0])

Proof of knowledge of a secret key `sk0` for Schnorr Signature

In [127]:
sk0 = [SchnorrDlog.Fp.random_element()]
pk0 = [SchnorrDlog.G * Integer(sk0[0])]

schnorr0 = SchnorrDlog(pk0)
pstate, commitment = schnorr0.prover_commit(sk0)
challenge = SchnorrDlog.Fp.random_element()
response = schnorr0.prover_response(pstate, challenge)
schnorr0.verifier(commitment, challenge, response)

True

An `AndComposition`, proving knowledge of two different secret keys `sk0` and `sk1`

In [128]:
sk1 = [SchnorrDlog.Fp.random_element()]
pk1 = [G * Integer(sk1[0])]

schnorr1 = SchnorrDlog(pk1)
schnorrAnd = SigmaAndComposition(schnorr0, schnorr1)
pstate, commitment = schnorrAnd.prover_commit((sk0, sk1))
challenge = SchnorrDlog.Fp.random_element()
response = schnorrAnd.prover_response(pstate, challenge)
schnorrAnd.verifier(commitment, challenge, response)

True